<a href="https://colab.research.google.com/github/Paras-Shirvale/A-B-Testing/blob/main/A_B_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import os

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
click_data = pd.read_csv("/content/ad_click_dataset.csv")

In [ ]:
Air_test_1 = pd.read_csv("/content/test1.csv")
Air_test_2 = pd.read_csv("/content/test2.csv")
Air_train = pd.read_csv("/content/train.csv")

In [ ]:
click_data.head()

,id,full_name,age,gender,device_type,ad_position,browsing_history,time_of_day,click
0,670,User670,22.0,NaN,Desktop,Top,Shopping,Afternoon,1
1,3044,User3044,NaN,Male,Desktop,Top,NaN,NaN,1
2,5912,User5912,41.0,Non-Binary,NaN,Side,Education,Night,1
3,5418,User5418,34.0,Male,NaN,NaN,Entertainment,Evening,1
4,9452,User9452,39.0,Non-Binary,NaN,NaN,Social Media,Morning,0


In [ ]:
Air_test_1.head()

,Unnamed: 0,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,3123,18/07/2004,21.00.00,"1,2",1067.0,-200.0,"9,0",938.0,102.0,825.0,99.0,1520.0,912.0,"29,7","24,8","1,0160",NaN,NaN
1,877,16/04/2004,07.00.00,"4,5",1657.0,523.0,"23,2",1384.0,352.0,579.0,109.0,2176.0,1600.0,"12,8","71,0","1,0428",NaN,NaN
2,3457,01/08/2004,19.00.00,"1,4",1037.0,-200.0,"8,0",900.0,75.0,817.0,95.0,1584.0,619.0,"33,1","32,7","1,6200",NaN,NaN
3,1494,12/05/2004,00.00.00,"1,7",1122.0,-200.0,"8,7",926.0,105.0,805.0,88.0,1619.0,1174.0,"16,9","58,8","1,1250",NaN,NaN
4,713,09/04/2004,11.00.00,"2,6",-200.0,262.0,"-200,0",-200.0,219.0,-200.0,121.0,-200.0,-200.0,-200,-200,-200,NaN,NaN


In [ ]:
Air_test_2.head()

,Unnamed: 0,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,8500,27/02/2005,22.00.00,"1,0",875.0,-200.0,"2,1",594.0,128.0,1079.0,105.0,793.0,451.0,"4,5","48,0","0,4085",NaN,NaN
1,8501,27/02/2005,23.00.00,"1,3",943.0,-200.0,"3,9",703.0,169.0,950.0,119.0,870.0,581.0,"4,3","48,6","0,4069",NaN,NaN
2,8502,28/02/2005,00.00.00,"1,6",947.0,-200.0,"3,8",697.0,215.0,913.0,150.0,878.0,698.0,"4,0","50,0","0,4115",NaN,NaN
3,8503,28/02/2005,01.00.00,"1,0",865.0,-200.0,"1,8",566.0,111.0,1119.0,94.0,797.0,423.0,"4,0","52,9","0,4338",NaN,NaN
4,8504,28/02/2005,02.00.00,"0,6",823.0,-200.0,"1,0",503.0,60.0,1268.0,56.0,755.0,332.0,"4,0","51,0","0,4200",NaN,NaN


In [ ]:
Air_train.head()

,Unnamed: 0,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,1849,26/05/2004,19.00.00,-200,1130.0,-200.0,"22,7",1368.0,-200.0,933.0,-200.0,1709.0,1269.0,"26,7","19,5","0,6754",NaN,NaN
1,2533,24/06/2004,07.00.00,"1,2",1030.0,-200.0,"6,9",851.0,102.0,824.0,68.0,1700.0,983.0,"21,9","57,0","1,4742",NaN,NaN
2,3047,15/07/2004,17.00.00,"3,2",1164.0,-200.0,"20,3",1306.0,259.0,648.0,198.0,1886.0,1218.0,"35,5","19,1","1,0888",NaN,NaN
3,805,13/04/2004,07.00.00,"3,9",1496.0,524.0,"19,1",1272.0,328.0,667.0,130.0,2011.0,1399.0,"11,0","64,2","0,8398",NaN,NaN
4,2962,12/07/2004,04.00.00,-200,780.0,-200.0,"1,8",568.0,24.0,1200.0,34.0,1331.0,501.0,"19,9","51,3","1,1803",NaN,NaN


In [ ]:
# Check for missing values
print(click_data.isnull().sum())

# Drop or fill missing values
click_data.dropna(inplace=True)  # Or use df.fillna() if you want to fill

id                     0
full_name              0
age                 4766
gender              4693
device_type         2000
ad_position         2000
browsing_history    4782
time_of_day         2000
click                  0
dtype: int64


In [ ]:
click_data.head()

,id,full_name,age,gender,device_type,ad_position,browsing_history,time_of_day,click
17,188,User188,56.0,Female,Tablet,Bottom,News,Morning,1
25,4890,User4890,43.0,Male,Tablet,Bottom,Education,Afternoon,1
33,4985,User4985,37.0,Male,Mobile,Top,News,Evening,0
52,9888,User9888,49.0,Male,Mobile,Top,News,Morning,1
102,8201,User8201,59.0,Female,Desktop,Bottom,Social Media,Morning,0


In [ ]:
# Step 2: Find unique values in 'gender' and 'ad_position'
unique_genders = click_data['gender'].unique()
unique_ad_positions = click_data['ad_position'].unique()

print(f"Unique genders: {unique_genders}")
print(f"Unique ad_positions: {unique_ad_positions}")

Unique genders: ['Female' 'Male' 'Non-Binary']
Unique ad_positions: ['Bottom' 'Top' 'Side']


In [ ]:
# Step 3: Map 'gender' and 'ad_position' columns
click_data['gender'] = click_data['gender'].map({'Male': 1, 'Female': 0, 'Non-Binary': -1})
click_data['ad_position'] = click_data['ad_position'].map({'Top': 0, 'Bottom': 1, 'Side': -1 })

# Make sure 'clicked' column is binary (0 or 1)
click_data['click'] = click_data['click'].astype(int)

In [ ]:
click_data.head()

,id,full_name,age,gender,device_type,ad_position,browsing_history,time_of_day,click
17,188,User188,56.0,0,Tablet,1,News,Morning,1
25,4890,User4890,43.0,1,Tablet,1,Education,Afternoon,1
33,4985,User4985,37.0,1,Mobile,0,News,Evening,0
52,9888,User9888,49.0,1,Mobile,0,News,Morning,1
102,8201,User8201,59.0,0,Desktop,1,Social Media,Morning,0


In [ ]:
# Split the dataset into two groups
group_A = click_data[click_data['ad_position'] == 0]
group_B = click_data[click_data['ad_position'] == 1]

In [ ]:
# Print the first few rows of each group to verify
print("Group A (ad_position = 0): Top")
print(group_A.head())

print("\nGroup B (ad_position = 1): Bottom")
print(group_B.head())

Group A (ad_position = 0): Top
       id full_name   age  gender device_type  ad_position browsing_history  \
33   4985  User4985  37.0       1      Mobile            0             News   
52   9888  User9888  49.0       1      Mobile            0             News   
158  3007  User3007  42.0       1     Desktop            0         Shopping   
204  8530  User8530  52.0       0      Mobile            0     Social Media   
231  4625  User4625  33.0      -1      Mobile            0             News   

    time_of_day  click  
33      Evening      0  
52      Morning      1  
158       Night      0  
204   Afternoon      1  
231     Morning      0  

Group B (ad_position = 1): Bottom
       id full_name   age  gender device_type  ad_position browsing_history  \
17    188   User188  56.0       0      Tablet            1             News   
25   4890  User4890  43.0       1      Tablet            1        Education   
102  8201  User8201  59.0       0     Desktop            1     Social Me

In [ ]:
!pip install --upgrade scipy

In [ ]:
import scipy
print(scipy.__version__)

1.14.1


In [ ]:
import statsmodels.api as sm

# Calculate the number of clicks and total observations for each group
successes = [group_A['click'].sum(), group_B['click'].sum()]
nobs = [len(group_A), len(group_B)]

# Perform the two-sample z-test
z_stat, p_val = sm.stats.proportions_ztest(successes, nobs)

# Print the results
print(f"Z-statistic: {z_stat}")
print(f"P-value: {p_val}")

# Interpret the results
# alpha = 0.05  # Significance level

# if p_val < alpha:
#     print("Reject the null hypothesis: There is a statistically significant difference in click-through rates between Group A and Group B.")
# else:
#     print("Fail to reject the null hypothesis: There is no statistically significant difference in click-through rates between Group A and Group B.")


Z-statistic: -1.1365075404030447
P-value: 0.2557442115851094


# Part 2: Covariate Shift Detection Using Air Quality Data

In [ ]:
# Load the datasets
train_df = pd.read_csv('/content/train.csv')
test1_df = pd.read_csv('/content/test1.csv')
test2_df = pd.read_csv('/content/test2.csv')

In [ ]:
train_df.head()

,Unnamed: 0,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,1849,26/05/2004,19.00.00,-200,1130.0,-200.0,"22,7",1368.0,-200.0,933.0,-200.0,1709.0,1269.0,"26,7","19,5","0,6754",NaN,NaN
1,2533,24/06/2004,07.00.00,"1,2",1030.0,-200.0,"6,9",851.0,102.0,824.0,68.0,1700.0,983.0,"21,9","57,0","1,4742",NaN,NaN
2,3047,15/07/2004,17.00.00,"3,2",1164.0,-200.0,"20,3",1306.0,259.0,648.0,198.0,1886.0,1218.0,"35,5","19,1","1,0888",NaN,NaN
3,805,13/04/2004,07.00.00,"3,9",1496.0,524.0,"19,1",1272.0,328.0,667.0,130.0,2011.0,1399.0,"11,0","64,2","0,8398",NaN,NaN
4,2962,12/07/2004,04.00.00,-200,780.0,-200.0,"1,8",568.0,24.0,1200.0,34.0,1331.0,501.0,"19,9","51,3","1,1803",NaN,NaN


In [ ]:
test1_df.head()

,Unnamed: 0,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,3123,18/07/2004,21.00.00,"1,2",1067.0,-200.0,"9,0",938.0,102.0,825.0,99.0,1520.0,912.0,"29,7","24,8","1,0160",NaN,NaN
1,877,16/04/2004,07.00.00,"4,5",1657.0,523.0,"23,2",1384.0,352.0,579.0,109.0,2176.0,1600.0,"12,8","71,0","1,0428",NaN,NaN
2,3457,01/08/2004,19.00.00,"1,4",1037.0,-200.0,"8,0",900.0,75.0,817.0,95.0,1584.0,619.0,"33,1","32,7","1,6200",NaN,NaN
3,1494,12/05/2004,00.00.00,"1,7",1122.0,-200.0,"8,7",926.0,105.0,805.0,88.0,1619.0,1174.0,"16,9","58,8","1,1250",NaN,NaN
4,713,09/04/2004,11.00.00,"2,6",-200.0,262.0,"-200,0",-200.0,219.0,-200.0,121.0,-200.0,-200.0,-200,-200,-200,NaN,NaN


In [ ]:
test2_df.head()

,Unnamed: 0,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,8500,27/02/2005,22.00.00,"1,0",875.0,-200.0,"2,1",594.0,128.0,1079.0,105.0,793.0,451.0,"4,5","48,0","0,4085",NaN,NaN
1,8501,27/02/2005,23.00.00,"1,3",943.0,-200.0,"3,9",703.0,169.0,950.0,119.0,870.0,581.0,"4,3","48,6","0,4069",NaN,NaN
2,8502,28/02/2005,00.00.00,"1,6",947.0,-200.0,"3,8",697.0,215.0,913.0,150.0,878.0,698.0,"4,0","50,0","0,4115",NaN,NaN
3,8503,28/02/2005,01.00.00,"1,0",865.0,-200.0,"1,8",566.0,111.0,1119.0,94.0,797.0,423.0,"4,0","52,9","0,4338",NaN,NaN
4,8504,28/02/2005,02.00.00,"0,6",823.0,-200.0,"1,0",503.0,60.0,1268.0,56.0,755.0,332.0,"4,0","51,0","0,4200",NaN,NaN


In [ ]:
from scipy.stats import ks_2samp

# Kolmogorov-Smirnov test for Test1 vs Train
ks_stat_test1, p_value_test1 = ks_2samp(train_df['NO2(GT)'], test1_df['NO2(GT)'])

# Kolmogorov-Smirnov test for Test2 vs Train
ks_stat_test2, p_value_test2 = ks_2samp(train_df['NO2(GT)'], test2_df['NO2(GT)'])

# Report the results
print(f"KS Statistic for Test1 vs Train: {ks_stat_test1}, p-value: {p_value_test1}")
print(f"KS Statistic for Test2 vs Train: {ks_stat_test2}, p-value: {p_value_test2}")

KS Statistic for Test1 vs Train: 0.0190625, p-value: 0.9721940612395358
KS Statistic for Test2 vs Train: 0.4075, p-value: 7.2019977111245e-96


In [ ]:
# Determine which dataset exhibits covariate shift
if p_value_test1 < 0.05:
    print("Test1 exhibits covariate shift relative to the training dataset.")
else:
    print("Test1 does not exhibit covariate shift relative to the training dataset.")

if p_value_test2 < 0.05:
    print("Test2 exhibits covariate shift relative to the training dataset.")
else:
    print("Test2 does not exhibit covariate shift relative to the training dataset.")


Test1 does not exhibit covariate shift relative to the training dataset.
Test2 exhibits covariate shift relative to the training dataset.


In [ ]:
import pandas as pd
from scipy.stats import ks_2samp

# Load the datasets
train_df = pd.read_csv('train.csv')
test1_df = pd.read_csv('test1.csv')
test2_df = pd.read_csv('test2.csv')

# Perform Kolmogorov-Smirnov test for NO2(GT) column
ks_stat_test1, p_value_test1 = ks_2samp(train_df['NO2(GT)'], test1_df['NO2(GT)'])
ks_stat_test2, p_value_test2 = ks_2samp(train_df['NO2(GT)'], test2_df['NO2(GT)'])

# Output the KS statistics and p-values
print(f"KS Statistic for Test1 vs Train: {ks_stat_test1}, p-value: {p_value_test1}")
print(f"KS Statistic for Test2 vs Train: {ks_stat_test2}, p-value: {p_value_test2}")

# Determine which dataset exhibits covariate shift
if p_value_test1 < 0.05:
    print("Test1 exhibits covariate shift relative to the training dataset.")
else:
    print("Test1 does not exhibit covariate shift relative to the training dataset.")

if p_value_test2 < 0.05:
    print("Test2 exhibits covariate shift relative to the training dataset.")
else:
    print("Test2 does not exhibit covariate shift relative to the training dataset.")


KS Statistic for Test1 vs Train: 0.0190625, p-value: 0.9721940612395358
KS Statistic for Test2 vs Train: 0.4075, p-value: 7.2019977111245e-96
Test1 does not exhibit covariate shift relative to the training dataset.
Test2 exhibits covariate shift relative to the training dataset.
